In [5]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

C:\Users\LGPC\anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\LGPC\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [6]:
review_data= pd.read_csv("Reviews.csv")
print(review_data.head(2))
print(len(review_data))
print('Unique Products')
print(len(review_data.groupby('ProductId')))
print('Unique Users')
print(len(review_data.groupby('UserId')))

   Id   ProductId          UserId ProfileName  HelpfulnessNumerator  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW  delmartian                     1   
1   2  B00813GRG4  A1D87F6ZCVE5NK      dll pa                     0   

   HelpfulnessDenominator  Score        Time                Summary  \
0                       1      5  1303862400  Good Quality Dog Food   
1                       0      1  1346976000      Not as Advertised   

                                                Text  
0  I have bought several of the Vitality canned d...  
1  Product arrived labeled as Jumbo Salted Peanut...  
568454
Unique Products
74258
Unique Users
256059


In [7]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [8]:
import nltk
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LGPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
review_data.dropna(axis = 0, how ='any',inplace=True) 






review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split())) 

print('-------Dataset --------')
print(review_data['Score'].value_counts())
print(len(review_data))
print('-------------------------')
max_review_data_sentence_length  = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]
df_sampled = df_short_reviews.groupby('Score').apply(lambda x: x.sample(n=20000)).reset_index(drop = True)

print('No of Short reviews')
print(len(df_short_reviews))



#all_sentences = train_data['text'].tolist() + test_data['text'].tolist()

-------Dataset --------
5    363111
4     80655
1     52264
3     42638
2     29743
Name: Score, dtype: int64
568411
-------------------------
No of Short reviews
373281


결측값이 있는 모든 행을 삭제합니다.
'clean_text' 함수를 사용하여 'Text' 열을 정리합니다.
'Num_words_text'라는 새 열을 DataFrame에 추가하고 각 정리된 텍스트의 단어 수를 포함시킵니다.
'Score' 열의 각 점수 값에 대한 리뷰 수, 데이터셋의 총 리뷰 수 및 구분자 몇 개를 출력합니다.
'Num_words_text' 열의 최대 문장 길이(단어 수)를 결정합니다.
짧은 리뷰(20~100 단어)를 필터링하기 위한 마스크를 생성합니다.
마스크를 기반으로 'df_short_reviews' DataFrame을 생성하고, 짧은 리뷰만 포함시킵니다.
'groupby' 및 'sample' 함수를 사용하여 'df_short_reviews' DataFrame에서 각 점수 범주에서 2만 개의 리뷰를 무작위로 추출합니다.
'df_short_reviews' 내의 짧은 리뷰 수를 출력합니다.

In [10]:
df_sampled

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,115371,B000F4F952,A30423CYQA2EKZ,Bluewater cruiser,1,7,1,1192233600,Poor taste compared to Ceylon Orange Pekoe,bought this before reading reviews here unfort...,38
1,419061,B000PZQUZO,A2XJV8WWLH4XX,"Dee from Gary, Indiana",3,3,1,1282608000,Terrible,calorie free carb free tell tried this twice w...,31
2,487236,B000PDWBKO,A3QGWIBB2M6D05,Erica Peterson,0,3,1,1318291200,popcorn is too salty,premeasured bags quick convenient popcorn salt...,53
3,273730,B002CJO8T6,ASFSYWVKD8NHZ,"Paul Sturgis ""paulsrealm""",1,5,1,1305504000,Amazon bait and switch,after only shipments part their automatic mont...,29
4,103816,B001EQ4IG6,A136FID8RG4INP,"Elizabeth ""gturner1""",0,1,1,1264032000,coffee values,because failed check over ounces store sells t...,34
...,...,...,...,...,...,...,...,...,...,...,...
99995,459008,B0031QKZO6,A27DHWJ275D678,Rev. George T. Cox,0,0,5,1329091200,Great Tasting Anti-Oxidant Tea,this outrageously great found green pomegranat...,21
99996,90914,B000FBM3PO,A2PF32UUERJXB8,"Susan Swan ""reads-curled-in-covers""",0,0,5,1187913600,"Oh, so very very very VERY good",love these cookies most cookies sweet these ho...,35
99997,540583,B001D0GV90,AU0GJWWEVFI4,Eric B. Brownstein,8,8,5,1203984000,The search is over,enjoy rich aromatic full bodied coffee with tr...,30
99998,471849,B000FPVLAS,A3IZQOD9Z3X707,T. D. Bourque,0,0,5,1350518400,Bob's Cherry Lumps,purchased these hard candies online because ca...,35


In [11]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_sampled['Text']=df_sampled['Text'].apply(remove_stopwords)
    

In [12]:
df_sampled['Text']

0        bought reading reviews unfortunately teas blen...
1        calorie free carb free tell tried twice wanted...
2        premeasured bags quick convenient popcorn salt...
3        shipments part automatic monthly subscription ...
4        failed check ounces store sells coffee less ha...
                               ...                        
99995    outrageously great found green pomegranate com...
99996    love cookies cookies sweet however since follo...
99997    enjoy rich aromatic full bodied coffee truly b...
99998    purchased hard candies online cant find locall...
99999    watched television explaining minimize andor a...
Name: Text, Length: 100000, dtype: object

In [15]:
!pip install spacy
!python -m spacy download en_core_web_md




[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [16]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [17]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [18]:
 
text_list=df_sampled['Text'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

calorie free carb free tell tried twice wanted really like horrible threw guess want sauce thats really going taste good youll take calories carbs stuff awful wasted money
['calorie', 'free', 'carb', 'free', 'tell', 'horrible', 'threw', 'guess', 'sauce', 'good', 'calorie', 'carb', 'stuff', 'awful', 'money']


In [19]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [20]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [21]:
lda_model.print_topics()

[(0,
  '0.041*"time" + 0.021*"candy" + 0.021*"product" + 0.021*"great" + 0.019*"year" + 0.016*"month" + 0.015*"last" + 0.014*"bread" + 0.014*"order" + 0.013*"good"'),
 (1,
  '0.041*"butter" + 0.036*"peanut" + 0.034*"cereal" + 0.025*"milk" + 0.022*"honey" + 0.017*"breakfast" + 0.016*"cake" + 0.010*"vitamin" + 0.010*"sugar" + 0.010*"crunchy"'),
 (2,
  '0.137*"coffee" + 0.034*"flavor" + 0.028*"good" + 0.022*"strong" + 0.018*"taste" + 0.017*"dark" + 0.016*"bean" + 0.016*"blend" + 0.015*"vanilla" + 0.013*"roast"'),
 (3,
  '0.084*"treat" + 0.063*"dog" + 0.046*"cat" + 0.043*"chicken" + 0.017*"pill" + 0.016*"small" + 0.014*"little" + 0.011*"training" + 0.010*"night" + 0.009*"dessert"'),
 (4,
  '0.077*"chocolate" + 0.044*"snack" + 0.028*"rice" + 0.027*"bar" + 0.023*"great" + 0.018*"good" + 0.018*"almond" + 0.016*"protein" + 0.015*"perfect" + 0.015*"cracker"'),
 (5,
  '0.112*"food" + 0.025*"product" + 0.024*"good" + 0.018*"organic" + 0.017*"ingredient" + 0.017*"year" + 0.015*"healthy" + 0.014*"b

In [23]:
!pip install --upgrade gensim pyLDAvis pandas


  Attempting uninstall: gensim
    Found existing installation: gensim 4.1.2
    Uninstalling gensim-4.1.2:
      Successfully uninstalled gensim-4.1.2


In [25]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary, n_jobs=1)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.077642 -0.070292       1        1  15.948394
8     -0.193036  0.096568       2        1  15.476752
7     -0.074890 -0.131326       3        1  12.673035
2     -0.064674 -0.094774       4        1  11.289657
5     -0.096885  0.040989       5        1  10.651303
0     -0.185100  0.161886       6        1  10.534433
6      0.042618 -0.113681       7        1   6.659373
4      0.096574 -0.171141       8        1   6.194324
1      0.343950 -0.047349       9        1   5.703366
3      0.209085  0.329119      10        1   4.869362, topic_info=          Term          Freq         Total Category  logprob  loglift
64      coffee  31028.000000  31028.000000  Default  30.0000  30.0000
136       food  23782.000000  23782.000000  Default  29.0000  29.0000
339  chocolate   9605.000000   9605.000000  Default  28.0000  28.0000
185     flavor  32276.000000  32276.000000  Default  27.0000  27.0000
466      treat   8194.000000   8194.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
80       smell    679.263915   2348.779077  Topic10  -4.9665   1.7816
134       fast    564.145066   1230.713301  Topic10  -5.1522   2.2422
352      piece    613.493811   3128.256359  Topic10  -5.0683   1.3931
68       large    562.144881   4623.432922  Topic10  -5.1557   0.9151
219    problem    529.066052   4980.307721  Topic10  -5.2164   0.7801

[538 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2506      8  0.998372  additional
98        5  0.998747       adult
1044      8  0.998706   afternoon
366       1  0.826382  aftertaste
366       4  0.173480  aftertaste
...     ...       ...         ...
124       6  0.326967        year
1246      8  0.998831       yeast
1788      1  0.998848      yogurt
1135      3  0.316505       yummy
1135      7  0.683217       yummy

[769 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 8, 3, 6, 1, 7, 5, 2, 4])

In [26]:

print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -8.828342780116756

Coherence Score:  0.4357555299288306


Perplexity (퍼플렉서티):

Perplexity는 LDA 모델의 학습 및 성능을 측정하는 지표 중 하나로, 모델이 주어진 데이터를 얼마나 잘 설명할 수 있는지를 나타냅니다. Perplexity는 확률 모델에서 말뭉치에 대한 예측의 "혼란도"를 측정합니다. Perplexity가 낮을수록 모델이 더 좋은 예측을 제공하며, 말뭉치의 "혼란" 정도가 낮다는 것을 의미합니다. 하지만 Perplexity는 특정 말뭉치에 특화되기 쉽고, 모델의 복잡성이나 토픽의 수에 따라 달라질 수 있습니다.

Coherence Score (일관성 점수):

Coherence Score는 LDA 토픽 모델의 주제들이 얼마나 일관성 있는지를 측정하는 지표입니다. 주제의 일관성은 해당 주제 내의 단어들이 함께 자주 등장하는 정도를 나타내며, 높은 일관성 점수는 모델이 의미 있는 주제를 잘 추출하고 있는 것을 나타냅니다. Coherence Score를 계산하기 위해서는 주제 내의 단어 간의 관련성을 측정하고, 주제 간의 관련성도 고려합니다. 높은 Coherence Score를 가진 모델은 보다 해석 가능하고 의미 있는 주제를 생성하는 경향이 있습니다.

이러한 평가 지표들은 LDA 모델의 성능과 일관성을 평가하며, 모델을 조정하거나 비교할 때 유용한 도구입니다. 하지만 어떤 평가 지표도 모델의 완전한 해석을 대신하지 못하므로, 다양한 평가 지표를 고려하여 종합적으로 모델을 평가하는 것이 중요합니다.

In [27]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [29]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)


KeyboardInterrupt: 

In [30]:
# Show graph
limit=50; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()# Print the coherence scores

NameError: name 'coherence_values' is not defined

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:

# Select the model and print the topics
optimal_model = model_list[7]
model_topics = optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=10)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, doc_term_matrix, dictionary, n_jobs=1)
vis